**Necessary Libraries**

In [ ]:
!apt-get install -y poppler-utils tesseract-ocr tesseract-ocr-ben
!pip install gdown pytesseract pdf2image faiss-cpu sentence-transformers transformers gradio

In [ ]:
import gdown
from pdf2image import convert_from_path
import pytesseract
import unicodedata
import re
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
from transformers import pipeline
import gradio as gr

**Load Dataset**

In [ ]:
file_id = "1YVPzZgXp6nyklR6SfNmLtV8tZJXqJtMH"
file_url = f"https://drive.google.com/uc?id={file_id}"
output = "HSC26-Bangla1st-Paper.pdf"
gdown.download(file_url, output, quiet=False)

**OCR from PDF**

In [ ]:
images = convert_from_path(output)
text = ""
for i, img in enumerate(images):
    print(f"OCR on page {i+1}/{len(images)}...")
    text += pytesseract.image_to_string(img, lang='ben') + "\n\n"

**Clean Text**

In [ ]:
def clean_bangla_text(text):
    text = unicodedata.normalize("NFC", text)
    text = re.sub(r"[^\S]+", " ", text)
    text = re.sub(r"[\u200c\u200d]", "", text)
    return text.strip()

text = clean_bangla_text(text)

**Extract MCQ Answer**

In [ ]:
def extract_mcq_answer_mapping(raw_text):
    page_41 = raw_text.split("৪১।")[1] if "৪১।" in raw_text else raw_text
    matches = re.findall(r"(\d+)\s+([কখগঘ])", page_41)
    return {int(num): letter for num, letter in matches}

mcq_answers = extract_mcq_answer_mapping(text)

**Chunk Text**

In [ ]:
paragraphs = [p.strip() for p in text.split("\n\n") if p.strip()]
chunk_size = 800
chunk_overlap = 100

chunks = []
for para in paragraphs:
    for i in range(0, len(para), chunk_size - chunk_overlap):
        chunks.append(para[i:i + chunk_size])

**Embedding**

In [ ]:
embedder = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
embeddings = embedder.encode(chunks, show_progress_bar=True)
embeddings = np.array(embeddings).astype("float32")

index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

**Retrieve**

In [ ]:
def retrieve_chunks(query, k=10):
    query_vec = embedder.encode([query]).astype("float32")
    D, I = index.search(query_vec, k)
    return [chunks[i] for i in I[0]]

**QA Model**

In [ ]:
qa_pipeline = pipeline("question-answering", model="deepset/xlm-roberta-base-squad2")

**Mapping Bangla Letter to Option**

In [ ]:
bangla_option_map = {
    "ক": 0, "খ": 1, "গ": 2, "ঘ": 3
}

**QA Answer Generator with MCQ support**

In [ ]:
def generate_answer(query):
    context = "\n\n".join(retrieve_chunks(query, k=10))
    result = qa_pipeline(question=query, context=context)
    answer = result["answer"]

    match = re.search(r"(\d+)।", query)
    if match:
        q_no = int(match.group(1))
        option_letter = mcq_answers.get(q_no)
        if option_letter:
            pattern = rf"{q_no}।(.+?)(?=\d+।|$)"
            mcq_block = re.search(pattern, text, re.DOTALL)
            if mcq_block:
                options = re.findall(r"\((ক|খ|গ|ঘ)\)\s*([^\(\)\n\r]+)", mcq_block.group(1))
                for opt, val in options:
                    if opt == option_letter:
                        return val.strip()
    return answer

**Live Demo App**

In [ ]:
def qa_interface(query):
    return generate_answer(query)

gr.Interface(fn=qa_interface,
             inputs=gr.Textbox(label="প্রশ্ন লিখুন (বাংলা বা English)"),
             outputs=gr.Textbox(label="উত্তর"),
             title="10MS RAG Pipeline").launch(share=True)